In [2]:
import pandas as pd
import unicodedata
import numpy as np
from geopy.geocoders import Nominatim

df = pd.read_csv('./input/main_lodz_full20230802-143112.csv',delimiter=';')


In [3]:
def type_clasificator(who:str):
    who = who.lower()
    if 'oferta prywatna' in who:
        return 'P'
    if 'inwestycja deweloperska' in who:
        return 'D'
    if 'biuro nieruchomości' in who:
        return 'N'
    return 'X'



In [4]:
df.head()


,date,link,name,where,price,perm,rooms,sqm,who
0,20230802,/pl/oferta/mieszkanie-54-m-lodz-ID4mkSG,"Mieszkanie, 54 m², Łódź","Mileszki, Widzew, Łódź, łódzkie",420 000 zł,7778 zł/m²,2 pokoje,54 m²,Oferta prywatna
1,20230802,/pl/oferta/3-pokojowe-mieszkanie-72m2-balkon-b...,3-pokojowe mieszkanie 72m2 + balkon Bezpośrednio,"ul. Mania, Zdrowie, Polesie, Łódź, łódzkie",Zapytaj o cenę,NaN,3 pokoje,72.39 m²,Esembla GroupInwestycja deweloperska
2,20230802,/pl/oferta/2-pokojowe-mieszkanie-38m2-balkon-I...,2-pokojowe mieszkanie 38m2 + balkon,"ul. Mania, Zdrowie, Polesie, Łódź, łódzkie",Zapytaj o cenę,NaN,2 pokoje,38.99 m²,Esembla GroupInwestycja deweloperska
3,20230802,/pl/oferta/3-pokojowe-mieszkanie-46m2-balkon-I...,3-pokojowe mieszkanie 46m2 + balkon,"ul. Hipoteczna, Stare Bałuty, Bałuty, Łódź, łó...",Zapytaj o cenę,NaN,3 pokoje,46.42 m²,Maxbud NoveoInwestycja deweloperska
4,20230802,/pl/oferta/4-pokojowe-mieszkanie-75m2-3-balkon...,4-pokojowe mieszkanie 75m2 + 3 balkony,"ul. Hipoteczna, Stare Bałuty, Bałuty, Łódź, łó...",Zapytaj o cenę,NaN,4 pokoje,75.45 m²,Maxbud NoveoInwestycja deweloperska


In [5]:
# unicode normalization
df['perm'] = df['perm'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan)
df['price'] = df['price'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan).str.replace(',','.')


In [6]:
# enrich fields
df['link'] = 'http://www.otodom.pl' + df['link']
df['sqm'] = df['sqm'].str.replace(' m²','').astype(float)
df['perm'] = df['perm'].str.replace('zł/m2','').str.replace(' ','').fillna(-1).astype(int)
df['price'] = df['price'].str.replace('zł','').str.replace('Zapytaj o cenę','-1').str.replace(' ','').fillna(-1).astype(float).astype(int)
df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)
# add new
df['id'] = df['link'].str[-7:]
df['type'] = df['who'].apply(type_clasificator)
# type adjustment
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# clean
df.replace({'who' : { 'Inwestycja deweloperska' : '', 'Biuro nieruchomości' : '' }}, inplace=True,regex=True)

C:\Users\krawc\AppData\Local\Temp\ipykernel_8776\2400223606.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)


In [68]:
#Geolocation
geolocator = Nominatim(user_agent="http")
def after_dot(string:str):
    return string[string.find('.')+1:]

def geo_nominatim(adress:str):
    try:
        adress = adress.split(', ')
        query_dict = {'street':after_dot(adress[0]),
            'city':adress[-2],
            # 'county':adress[-3],
            'state':adress[-1],
            'country':'pl'}
        nominatim = geolocator.geocode(query_dict,country_codes='pl',exactly_one=True,addressdetails=True)
        
        # names=['ulica','osiedle','rejon','dzielnica','miasto','wojewodztwo','kod','kraj']
        # location_string = nominatim.raw['display_name'].split(', ')
        # ndict = dict(zip(names,location_string))
        ndict = nominatim.raw['address']

        return (nominatim.latitude,nominatim.longitude,ndict)
    except:
        pass

def add_geo(series):
    la,lo,d = geo_nominatim(series['where'])
    series['latitude'],series['longitude'],series['neighbourhood'],series['suburb'] = la,lo,d['neighbourhood'],d['suburb']
    return series

(51.7979587,
 19.432959,
 {'road': 'Hipoteczna',
  'neighbourhood': 'Berlinek',
  'suburb': 'Łódź-Bałuty',
  'city': 'Łódź',
  'state': 'województwo łódzkie',
  'ISO3166-2-lvl4': 'PL-10',
  'postcode': '91-338',
  'country': 'Polska',
  'country_code': 'pl'})

In [74]:
df2 = df.sample(5)

In [75]:

df2 = df2.apply(add_geo,axis=1)
df2


,date,link,name,where,price,perm,rooms,sqm,who,id,type,latitude,longitude,neighbourhood,suburb
386,2023-08-02,http://www.otodom.pl/pl/oferta/2-pokoje-w-cent...,2 Pokoje w Centrum - Uniwersytet ‼️ Ten rok ‼️,"ul. dr. Stefana Kopcińskiego, Stary Widzew, Wi...",368740,8950,2,41.20,RYNEK PIERWOTNY,ID4mura,N,51.775723,19.482244,Radiostacja,Łódź-Śródmieście
4352,2023-08-02,http://www.otodom.pl/pl/oferta/mieszkanie-na-d...,Mieszkanie na drugim piętrze super okolicy .,"ul. ks. Hugona Kołłątaja, Fabryczna, Śródmieśc...",370000,6444,2,57.42,Metrohouse Franchise S.A.,ID4kbJD,N,51.779890,19.475848,Śródmieście-Wschód,Łódź-Śródmieście
4136,2023-08-02,http://www.otodom.pl/pl/oferta/2-pok-loft-na-t...,2-pok. Loft na Tymienieckiego 25D,"ul. ks. bp. Wincentego Tymienieckiego, Księży ...",499851,9900,2,50.49,AJ Promotion,ID4kO1u,N,51.748744,19.461717,Katedralna,Łódź-Śródmieście
1987,2023-08-02,http://www.otodom.pl/pl/oferta/mieszkanie-2-po...,Mieszkanie 2-pokojowe na Karolewie,"Karolew, Polesie, Łódź, łódzkie",286000,7666,2,37.31,House Factory,ID4mbBL,N,51.747726,19.420901,Karolew,Łódź-Polesie
464,2023-08-02,http://www.otodom.pl/pl/oferta/4-pokoje-po-rem...,"4 Pokoje Po Remoncie, Ul. Brzóski 34","ul. ks. Stanisława Brzóski, Stare Bałuty, Bału...",585000,7519,4,77.80,HAPPY HOME MACIEJ IRACKI,ID4lmqk,N,51.790529,19.438682,Osiedle Pojezierska,Łódź-Bałuty
